In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os

import mechanism as mc
import filtering

%matplotlib inline
np.set_printoptions(precision=4)

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["mathtext.fontset"] = "stix" 
plt.rcParams["font.size"] = 12
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True
del matplotlib.font_manager.weight_dict['roman']
matplotlib.font_manager._rebuild()

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['image.cmap'] = 'viridis'

plt.rcParams["errorbar.capsize"] = 2.0 # error bar

In [ ]:
def MAE(x, y):
    return np.mean(np.abs(x-y))

In [ ]:
def get_signal(freq, noise_flag=True):
    T = 10000
    t = np.arange(0, T, 1/freq)
    a = 200
    b = 500
    omega = 0.01
    y = a * np.sin(omega * t) + b + 0.1 * t
    x = y
    if noise_flag:
        noise = 100 * np.random.randn(t.shape[0])
        x = y + noise
    return x, t, y

## MAEs with Signal with observation noise

In [ ]:
trial = 1000
gaussian_res = ([], [])
dft_res = ([], [])
ss_res = ([], [])
ssf_res = ([], [])

freqs = 1/np.array([1, 2, 4, 8, 16, 32])


I = 100
l2_sens = np.sqrt(I)

for freq in freqs:
    x, t, y = get_signal(freq, noise_flag=True)
    T = len(x)
    print(T)
    k = 10 / (1+np.log2(1/freq)/4)

    std = 10 / (1+np.log2(1/freq)/4)

    h = filtering.get_h('gaussian', T, std=std)
    A = filtering.get_circular(h)
    L = sum(h**2)
    sr = mc.srank_circular(h)

    eps = 0.5
    delta = 10**-4

    target = y  # target is one before noise is added
    res_z = []
    res_zss = []
    res_zfss = []
    res_zdft = []
    res_zfast = []
    for _ in range(trial):
        z = mc.gaussian(x, eps, delta, l2_sens)
        zss = mc.ss_gaussian(x, eps, delta, I, k, interpolate_kind='linear')
        zfss = mc.ssf_gaussian(x, A, eps, delta, l2_sens, k, sr=sr, L=L, interpolate_kind='linear')
        zdft = mc.dft_gaussian(x, eps, delta, l2_sens, k=30)
        res_z.append(MAE(target, z))
        res_zss.append(MAE(target, zss))
        res_zfss.append(MAE(target, zfss))
        res_zdft.append(MAE(target, zdft))
    gaussian_res[0].append(np.mean(res_z))
    gaussian_res[1].append(np.std(res_z))
    dft_res[0].append(np.mean(res_zdft))
    dft_res[1].append(np.std(res_zdft))
    ss_res[0].append(np.mean(res_zss))
    ss_res[1].append(np.std(res_zss))
    ssf_res[0].append(np.mean(res_zfss))
    ssf_res[1].append(np.std(res_zfss))

plt.figure(figsize=(7,5))
plt.xscale('log')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('MAE', fontsize=15)
plt.xlabel('Relative Sampling Frequency', fontsize=15)
plt.errorbar(freqs, gaussian_res[0], yerr=gaussian_res[1], label='Gaussian')
plt.errorbar(freqs, dft_res[0], yerr=dft_res[1], label='DFT')
plt.errorbar(freqs, ss_res[0], yerr=ss_res[1], label='Ours w/o Filter')
plt.errorbar(freqs, ssf_res[0], yerr=ssf_res[1], label='Ours w/ Filter')

plt.legend()

## MAEs with Signal without observation noise

In [ ]:
trial = 1000
gaussian_res = ([], [])
dft_res = ([], [])
ss_res = ([], [])
ssf_res = ([], [])

freqs = 1/np.array([1, 2, 4, 8, 16, 32])


I = 100
l2_sens = np.sqrt(I)

for freq in freqs:
    x, t, y = get_signal(freq, noise_flag=False)
    T = len(x)
    k = 10 / (1+np.log2(1/freq)/4)
    std = 10 / (1+np.log2(1/freq)/4)

    h = filtering.get_h('gaussian', T, std=std)
    A = filtering.get_circular(h)
    L = sum(h**2)
    sr = mc.srank_circular(h)

    eps = 0.5
    delta = 10**-4

    target = y
    res_z = []
    res_zss = []
    res_zfss = []
    res_zdft = []
    for _ in range(trial):
        z = mc.gaussian(x, eps, delta, l2_sens)
        zss = mc.ss_gaussian(x, eps, delta, I, k, interpolate_kind='linear')
        zfss = mc.ssf_gaussian(x, A, eps, delta, l2_sens, k, sr=sr, L=L, interpolate_kind='linear')
        zdft = mc.dft_gaussian(x, eps, delta, l2_sens, k=30)
        res_z.append(MAE(target, z))
        res_zss.append(MAE(target, zss))
        res_zfss.append(MAE(target, zfss))
        res_zdft.append(MAE(target, zdft))
    gaussian_res[0].append(np.mean(res_z))
    gaussian_res[1].append(np.std(res_z))
    dft_res[0].append(np.mean(res_zdft))
    dft_res[1].append(np.std(res_zdft))
    ss_res[0].append(np.mean(res_zss))
    ss_res[1].append(np.std(res_zss))
    ssf_res[0].append(np.mean(res_zfss))
    ssf_res[1].append(np.std(res_zfss))

plt.figure(figsize=(7,5))
plt.xscale('log')
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('MAE', fontsize=15)
plt.xlabel('Relative Sampling Frequency', fontsize=15)
plt.errorbar(freqs, gaussian_res[0], yerr=gaussian_res[1], label='Gaussian')
plt.errorbar(freqs, dft_res[0], yerr=dft_res[1], label='DFT')
plt.errorbar(freqs, ss_res[0], yerr=ss_res[1], label='Ours w/o Filter')
plt.errorbar(freqs, ssf_res[0], yerr=ssf_res[1], label='Ours w/ Filter')

plt.legend()

## Outputs with Signal with observation noise

In [ ]:
freq = 1
x, t, y = get_signal(freq, noise_flag=True)

T = len(x)
print(T)
k = 10 / (1+np.log2(1/freq)/4)

std = 10 / (1+np.log2(1/freq)/4)

h = filtering.get_h('gaussian', T, std=std)
A = filtering.get_circular(h)
L = sum(h**2)
sr = mc.srank_circular(h)

eps = 0.5
delta = 10**-4
I = 100
l2_sens = np.sqrt(I)

z = mc.gaussian(x, eps, delta, l2_sens)
zss = mc.ss_gaussian(x, eps, delta, I, k, interpolate_kind='linear')
zfss = mc.ssf_gaussian(x, A, eps, delta, l2_sens, k, sr=sr, L=L, interpolate_kind='linear')
zdft = mc.dft(x, eps, l2_sens, k=30)
alpha = 0.7

plt.figure(figsize=(18,6))
ax1 = plt.subplot(141)
plt.plot(y, label='Raw Synth Signal', linewidth=5, ls='-')
plt.plot(z, label='Gaussian', alpha=alpha)
plt.legend()
plt.xlim((0,3000))
plt.ylim((0,1600))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax2 = plt.subplot(142, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zdft, label='DFT', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax3 = plt.subplot(143, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zss, label='Ours w/o Filter', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax4 = plt.subplot(144, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zfss, label='Ours w/ Filter', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()

## Outputs with Signal without observation noise

In [ ]:
freq = 1
x, t, y = get_signal(freq, noise_flag=False)
T = len(x)
print(T)
k = 10 / (1+np.log2(1/freq)/4)

std = 10 / (1+np.log2(1/freq)/4)

h = filtering.get_h('gaussian', T, std=std)
A = filtering.get_circular(h)
L = sum(h**2)
sr = mc.srank_circular(h)

eps = 0.5
delta = 10**-4
I = 100  # base T = 10000
l2_sens = np.sqrt(I)

z = mc.gaussian(x, eps, delta, l2_sens)
zss = mc.ss_gaussian(x, eps, delta, I, k, interpolate_kind='linear')
zfss = mc.ssf_gaussian(x, A, eps, delta, l2_sens, k, sr=sr, L=L, interpolate_kind='linear')
zdft = mc.dft(x, eps, l2_sens, k=30)
alpha = 0.7

plt.figure(figsize=(18,6))
ax1 = plt.subplot(141)
plt.plot(y, label='Raw Synth Signal', linewidth=5, ls='-')
plt.plot(z, label='Gaussian', alpha=alpha)
plt.legend()
plt.xlim((0,3000))
plt.ylim((0,1600))
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax2 = plt.subplot(142, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zdft, label='DFT', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax3 = plt.subplot(143, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zss, label='Ours w/o Filter', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()
ax4 = plt.subplot(144, sharex=ax1, sharey=ax1)
plt.plot(y, linewidth=5, ls='-')
plt.plot(zfss, label='Ours w/ Filter', alpha=alpha)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time Step', fontsize=15)
plt.legend()